In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Activation, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.applications import ResNet50V2, ResNet101V2, DenseNet121, VGG16, VGG19
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1, 2, 3'

2022-05-07 19:03:02.376245: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2022-05-07 19:03:02.376331: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64/usr/local/cuda-10.2/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2022-05-07 19:03:02.376340: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
def load_CIFAR10(num_class):
    """
    Load CIFAR10 dataset and data preprocessing.
    :param data_dir: Path where to find the data file.
    :return: Tuple of numpy arrays: (x_train, y_train), (x_test, y_test).
    """
    # Initialize Data
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    x_train = x_train / 255.0
    x_test = x_test / 255.0
    
    y_train = tf.keras.utils.to_categorical(y_train, num_class)
    y_test = tf.keras.utils.to_categorical(y_test, num_class)
    
    return (x_train, y_train), (x_test, y_test)

(x, y), (x_test, y_test) = load_CIFAR10(10)

In [3]:
x.shape

(50000, 32, 32, 3)

In [4]:
def get_Resnet50(input_shape, num_classes):
    """Define and return the ResNet50V2 model.
    Args:
        input_shape: a tensor defined the data shape.
        num_classes: the number of classes.
    Returns: 
        a keras model object named ResNet50V2.
    """
    model = tf.keras.Sequential([
        ResNet50V2(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape),
        GlobalAveragePooling2D(),
        Dense(num_classes),
        Activation('softmax')
    ])
    model.summary()
    
    return model

model = get_Resnet50(x[0].shape, num_classes=10)

2022-05-07 19:03:05.136046: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-05-07 19:03:05.156652: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 0 with properties: 
pciBusID: 0000:41:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2022-05-07 19:03:05.157862: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 1 with properties: 
pciBusID: 0000:45:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2022-05-07 19:03:05.159036: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1555] Found device 2 with properties: 
pciBusID: 0000:47:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 23,585,290
Trainable params: 23,539,850
Non-trainable params: 45,440
_________________________________________________________________


In [5]:
def get_Shadow50(input_shape, num_classes):
    """Define and return the ResNet50V2 model.
    Args:
        input_shape: a tensor defined the data shape.
        num_classes: the number of classes.
    Returns: 
        a keras model object named ResNet50V2.
    """
    model = tf.keras.Sequential([
        ResNet50V2(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape),
        GlobalAveragePooling2D(),
        Dense(num_classes),
        Activation('softmax')
    ])
    model.summary()
    
    return model

shadow = get_Shadow50(x[0].shape, num_classes=10)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 1, 1, 2048)        23564800  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 23,585,290
Trainable params: 23,539,850
Non-trainable params: 45,440
_________________________________________________________________


In [6]:
def entropy(c):
    if(len(c) > 0):
        result = 0
    for i in range(len(c[0])):
        result += (-c[0, i]) * np.log2(c[0, i])
    return result
   

In [7]:
entropys = [0, 0, 0, 0, 0]
out = []
entropyaaa = []
for i in range(5): 
    i = i % 5   
    x_train, y_train = x[i:i+5000], y[i:i+5000]
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.categorical_crossentropy,
                metrics = ["accuracy"])

    model.fit(x_train, y_train,
                epochs=10,
                batch_size=64,
                shuffle=True,
                verbose=1)

    for j in range(5000):
        x1 = x[j].reshape(1, 32, 32, 3)
        out = model.predict(x1)
        entropys[i] += entropy(out)
    entropyaaa.append(entropys)

Train on 5000 samples
Epoch 1/10


2022-05-07 19:03:21.602356: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-05-07 19:03:21.944507: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2022-05-07 19:03:23.157358: W tensorflow/stream_executor/gpu/redzone_allocator.cc:312] Not found: ./bin/ptxas not found
Relying on driver to perform ptx compilation. This message will be only logged once.


5000/5000 [==============================] - 14s 3ms/sample - loss: 2.3991 - accuracy: 0.1518
Epoch 2/10
5000/5000 [==============================] - 4s 721us/sample - loss: 2.0456 - accuracy: 0.2828
Epoch 3/10
5000/5000 [==============================] - 3s 676us/sample - loss: 1.7326 - accuracy: 0.4312
Epoch 4/10
5000/5000 [==============================] - 4s 714us/sample - loss: 1.4025 - accuracy: 0.5300
Epoch 5/10
5000/5000 [==============================] - 4s 748us/sample - loss: 1.1202 - accuracy: 0.6256
Epoch 6/10
5000/5000 [==============================] - 4s 772us/sample - loss: 0.8911 - accuracy: 0.7040
Epoch 7/10
5000/5000 [==============================] - 4s 759us/sample - loss: 0.6629 - accuracy: 0.7926
Epoch 8/10
5000/5000 [==============================] - 4s 725us/sample - loss: 0.4868 - accuracy: 0.8502
Epoch 9/10
5000/5000 [==============================] - 4s 711us/sample - loss: 0.3711 - accuracy: 0.8892
Epoch 10/10
5000/5000 [==============================] - 4

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log2
  """
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """


Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 12s 2ms/sample - loss: 0.0544 - accuracy: 0.9818
Epoch 2/10
5000/5000 [==============================] - 4s 724us/sample - loss: 0.0463 - accuracy: 0.9882
Epoch 3/10
5000/5000 [==============================] - 4s 778us/sample - loss: 0.0360 - accuracy: 0.9880
Epoch 4/10
5000/5000 [==============================] - 4s 735us/sample - loss: 0.0323 - accuracy: 0.9896
Epoch 5/10
5000/5000 [==============================] - 4s 732us/sample - loss: 0.0313 - accuracy: 0.9912
Epoch 6/10
5000/5000 [==============================] - 4s 728us/sample - loss: 0.0307 - accuracy: 0.9900
Epoch 7/10
5000/5000 [==============================] - 4s 759us/sample - loss: 0.0375 - accuracy: 0.9896
Epoch 8/10
5000/5000 [==============================] - 4s 750us/sample - loss: 0.0279 - accuracy: 0.9906
Epoch 9/10
5000/5000 [==============================] - 4s 716us/sample - loss: 0.0253 - accuracy: 0.9926
Epoch 10/10
5000/5000 [==

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log2
  """
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """


Train on 5000 samples
Epoch 1/10
5000/5000 [==============================] - 13s 3ms/sample - loss: 0.0406 - accuracy: 0.9862
Epoch 2/10
5000/5000 [==============================] - 4s 718us/sample - loss: 0.0299 - accuracy: 0.9910
Epoch 3/10
5000/5000 [==============================] - 4s 730us/sample - loss: 0.0222 - accuracy: 0.9938
Epoch 4/10
5000/5000 [==============================] - 4s 728us/sample - loss: 0.0261 - accuracy: 0.9922
Epoch 5/10
5000/5000 [==============================] - 4s 758us/sample - loss: 0.0206 - accuracy: 0.9940
Epoch 6/10
5000/5000 [==============================] - 4s 727us/sample - loss: 0.0474 - accuracy: 0.9894
Epoch 7/10
5000/5000 [==============================] - 4s 770us/sample - loss: 0.0312 - accuracy: 0.9902
Epoch 8/10
5000/5000 [==============================] - 4s 766us/sample - loss: 0.0276 - accuracy: 0.9916
Epoch 9/10
5000/5000 [==============================] - 4s 755us/sample - loss: 0.0279 - accuracy: 0.9914
Epoch 10/10
5000/5000 [==

/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log2
  """
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in float_scalars
  """


net = get_Resnet50(x[0].shape, num_classes=10)
for i in range(5): 
    i = i % 5 
    i += 25000  
    x_train, y_train = x[i:i+5000], y[i:i+5000]
    net.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6),
                loss=tf.keras.losses.categorical_crossentropy,
                metrics = ["accuracy"])

    net.fit(x_train, y_train,
                epochs=10,
                batch_size=64,
                shuffle=True,
                verbose=1)

In [8]:
out

array([[1.6824601e-13, 4.6230255e-12, 7.1101756e-11, 6.5110444e-13,
        1.4588138e-09, 3.3469640e-11, 1.0000000e+00, 5.2383086e-14,
        1.5554744e-14, 2.4789190e-13]], dtype=float32)

In [9]:
entropys

[1477.1250301718876, 200.5574101187213, nan, nan, nan]

In [10]:
for i in range(len(entropys)):
    print("The {}st entopys is: {}".format(i, entropys[i]))

The 0st entopys is: 1477.1250301718876
The 1st entopys is: 200.5574101187213
The 2st entopys is: nan
The 3st entopys is: nan
The 4st entopys is: nan


In [11]:
# Training a Attack model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

attack = RandomForestClassifier(n_estimators=50)
x_train , y_train = model.predict(x), np.zeros(shape=(60000, 1))
for i in range(5000, 60000):
    y_train[i] = 1
# print(y_train)
print(x_train.shape, y_train.shape)
"""x_t = np.vstack((x_train[:2500], x_train[5000: 7500]))
x_t = np.vstack((x_t, x_train[10000: 12500]))
x_t = np.vstack((x_t, x_train[15000: 17500]))
x_t = np.vstack((x_t, x_train[20000: 22500]))
x_t = np.vstack((x_t, x_train[25000: 27500]))
y_t = np.vstack((y_train[:2500], y_train[5000: 7500]))
y_t = np.vstack((y_t, y_train[10000: 12500]))
y_t = np.vstack((y_t, y_train[15000: 17500]))
y_t = np.vstack((y_t, y_train[20000: 22500]))
y_t = np.vstack((y_t, y_train[25000: 27500]))"""
attack.fit(x_train[2500:5000], y_train[2500:5000])
y_pred = attack.predict(model.predict(x[:5000])).reshape((5000, 1))
y_pred2 = attack.predict(model.predict(x_test)).reshape((10000, 1))
y_pred = np.vstack((y_pred, y_pred2))
print(classification_report(y_true=y_train[0:15000], y_pred=y_pred))

(50000, 10) (60000, 1)


/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/ipykernel_launcher.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


              precision    recall  f1-score   support

         0.0       0.33      1.00      0.50      5000
         1.0       0.00      0.00      0.00     10000

    accuracy                           0.33     15000
   macro avg       0.17      0.50      0.25     15000
weighted avg       0.11      0.33      0.17     15000



/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/kolla/GPU2/anaconda3/envs/blindmi/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [12]:
from sklearn.cluster import KMeans

result = []
for k in range(2, 10):
    kmodel = KMeans(n_clusters=k).fit(x_train[27500:32500])
    y_pred =kmodel.predict(model.predict(x)).reshape((50000, 1))
    y_pred2 = kmodel.predict(model.predict(x_test)).reshape((10000, 1))
    y_pred = np.vstack((y_pred, y_pred2))
    print(classification_report(y_true=y_train[0:60000], y_pred=y_pred))

: 

: 